In [1]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy")

<Experiment: artifact_location='mlflow-artifacts:/374362034103955121', creation_time=1741686562632, experiment_id='374362034103955121', last_update_time=1741686562632, lifecycle_stage='active', name='DSPy', tags={}>

In [2]:
mlflow.dspy.autolog()

/Users/aidand/dev/hello-dspy/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from dotenv import load_dotenv

load_dotenv()


True

In [23]:
import dspy

llama3b = dspy.LM('fireworks_ai/accounts/fireworks/models/llama-v3p2-3b-instruct', temperature=0.7)
gpt4o = dspy.LM('openai/gpt-4o', temperature=0.7)

dspy.configure(lm=llama3b)

In [5]:
import random
from dspy.datasets import DataLoader

kwargs = dict(fields=("claim", "supporting_facts", "hpqa_id", "num_hops"), input_keys=("claim",))
hover = DataLoader().from_huggingface(dataset_name="hover-nlp/hover", split="train", trust_remote_code=True, **kwargs)

hpqa_ids = set()
hover = [
    dspy.Example(claim=x.claim, titles=list(set([y["key"] for y in x.supporting_facts]))).with_inputs("claim")
    for x in hover
    if x["num_hops"] == 3 and x["hpqa_id"] not in hpqa_ids and not hpqa_ids.add(x["hpqa_id"])
]

random.Random(0).shuffle(hover)
trainset, devset, testset = hover[:100], hover[100:200], hover[650:]

Generating test split: 100%|██████████| 4000/4000 [00:00<00:00, 68392.45 examples/s]


In [6]:
example = trainset[0]

print("Claim:", example.claim)
print("Pages that must be retrieved:", example.titles)

Claim: This director is known for his work on Miss Potter. The Academy of Motion Picture Arts and Sciences presents the award in which he was nominated for his work in "Babe".
Pages that must be retrieved: ['Miss Potter', 'Chris Noonan', 'Academy Award for Best Director']


In [7]:
DOCS = {}

def search(query: str, k: int) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=k)
    results = [x['text'] for x in results]

    for result in results:
        title, text = result.split(" | ", 1)
        DOCS[title] = text

    return results

In [8]:
def search_wikipedia(query: str) -> list[str]:
    """Returns top-5 results and then the titles of the top-5 to top-30 results."""

    topK = search(query, 30)
    titles, topK = [f"`{x.split(' | ')[0]}`" for x in topK[5:30]], topK[:5]
    return topK + [f"Other retrieved pages have titles: {', '.join(titles)}."]

def lookup_wikipedia(title: str) -> str:
    """Returns the text of the Wikipedia page, if it exists."""

    if title in DOCS:
        return DOCS[title]

    results = [x for x in search(title, 10) if x.startswith(title + " | ")]
    if not results:
        return f"No Wikipedia page found for title: {title}"
    return results[0]

In [9]:
instructions = "Find all Wikipedia titles relevant to verifying (or refuting) the claim."
signature = dspy.Signature("claim -> titles: list[str]", instructions)
react = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=20)

In [19]:
react(claim="David Gregory was born in 1625.").titles[:3]

[]

Trace(request_id=a2a603dfc6e848cb8b43a1e13551ede7)

In [20]:
dspy.inspect_history(n=1)





[2025-03-12T18:38:23.559367]

System message:

Your input fields are:
1. `claim` (str)
2. `trajectory` (str)

Your output fields are:
1. `reasoning` (str)
2. `titles` (list[str])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## claim ## ]]
{claim}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## titles ## ]]
{titles}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Find all Wikipedia titles relevant to verifying (or refuting) the claim.


User message:

[[ ## claim ## ]]
David Gregory was born in 1625.

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
Searching for information about David Gregory's birth.

[[ ## tool_name_0 ## ]]
search_wikipedia

[[ ## tool_args_0 ## ]]
{"query": {"type": "string"}}

[[ ## observation_0 ## ]]
Failed to execute: Arg quer

In [21]:
def top5_recall(example, pred, trace=None):
    gold_titles = example.titles
    recall = sum(x in pred.titles[:5] for x in gold_titles) / len(gold_titles)

    # If we're "bootstrapping" for optimization, return True if and only if the recall is perfect.
    if trace is not None:
        return recall >= 1.0
    
    # If we're just doing inference, just measure the recall.
    return recall

evaluate = dspy.Evaluate(devset=devset, metric=top5_recall, num_threads=16, display_progress=True, display_table=5)

In [24]:
def safe_react(claim: str):
    try:
        return react(claim=claim)
    except Exception as e:
        return dspy.Prediction(titles=[])

evaluate(safe_react)

Average Metric: 25.00 / 100 (25.0%): 100%|██████████| 100/100 [04:23<00:00,  2.63s/it]

2025/03/12 18:43:58 INFO dspy.evaluate.evaluate: Average Metric: 24.99999999999999 / 100 (25.0%)


,claim,example_titles,trajectory,reasoning,pred_titles,top5_recall
0,The Church of England's movement that inspired the Trinity Episcop...,"[Oxford Movement, Trinity Episcopal Church (Houghton, Michigan), S...","{'thought_0': ""The claim mentions the Church of England and the Tr...",The claim mentions the Church of England and the Trinity Episcopal...,"['John Lettice', 'John Kettlewell', 'John Purchas', 'Joachim Lütke...",
1,"Red, White & Crüe and this athlete both fight. The french fighter ...","[Mike Tyson, Bobby Stewart, Red, White &amp; Crüe]","{'thought_0': 'The french fighter is likely Patrick Surtain', 'too...",The reasoning is that the tool was unable to find reliable informa...,"['Billy Stewart', 'Jean-Patrick Lescarboura', 'Simon Patrick Stewa...",
2,The writer/director/actor from Glen or Glenda and Fernand Rivers s...,"[Fernand Rivers, Ed Wood, Glen or Glenda]",{'thought_0': 'The writer/director/actor from Glen or Glenda and F...,The writer/director/actor from Glen or Glenda and Fernand Rivers i...,['The Incredibly Strange Creatures Who Stopped Living and Became M...,
3,The film by Sandi Sissel was released before The End of Suburbia.,"[Sandi Sissel, Chicken Ranch (film), The End of Suburbia]",{'thought_0': 'Search for the director of The End of Suburbia to s...,The director of The End of Suburbia may have also directed the fil...,"['Sandi Sissel', 'Escape from Suburbia', 'Suburb (1951 film)', 'Su...",✔️ [0.333]
4,The actor who played captain hook in the live production with Tayl...,"[Taylor Louderman, Christopher Walken, Peter Pan Live!]",{'thought_0': 'The actor who played captain hook in the live produ...,The reasoning behind this response is that none of the provided to...,"['Cyril Ritchard', 'Captain James Hook', 'E. Holman Clark', 'Peter...",


25.0

[Trace(request_id=875a9051f7e749f1ba7aaf8200dc1ac6), Trace(request_id=090ffb3a1a5b45108fb5aa1f07b1716d), Trace(request_id=91027bc83e214a169221ae7400cb04b9), Trace(request_id=be4ed3e01bc145deb3415efde21b19b2), Trace(request_id=74c2819f278844c1ac0d27ca45e7f14d), Trace(request_id=cc1665a7fcfc431ca5e358e59a90b3bc), Trace(request_id=a9443893823f4e51b39714f803cfcf2e), Trace(request_id=3a130543ee004d7daa6a77c5310a76ee), Trace(request_id=0c3a3ffebb394a90af54edf918e1df9b), Trace(request_id=5529cbd922d54eb7882406c5394966cc)]

In [ ]:
dspy.inspect_history(n=1)